# DEEPCAMP_P2C_PREDICTIONS
### Objectif_Notebook : Machine Learning avec prédictions Météo France

# PREPARATION DES DONNEES

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import os
import xgboost as xgb
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:

#Données apprentissage
df_station_train = pd.read_csv("C:/Users/MSI GT 70 APACHE PRO/Desktop/Kaggle/DataPerHourV2.csv")
df_station_test = pd.read_csv("C:/Users/MSI GT 70 APACHE PRO/Desktop/Kaggle/XtestV1.csv")
#df_baseline_observation = pd.read_csv("../input/defi-ia-2022/Train/Train/Baselines/Baseline_observation_train.csv")
df_y_train = pd.read_csv("C:/Users/MSI GT 70 APACHE PRO/Desktop/Kaggle/Y_train.csv")
#df_baseline_forecast   = pd.read_csv("../input/defi-ia-2022/Train/Train/Baselines/Baseline_forecast_train.csv")
df_stations_coordinate = pd.read_csv("C:/Users/MSI GT 70 APACHE PRO/Desktop/Kaggle/stations_coordinates.csv")
baseline_forecast = pd.read_csv('C:/Users/MSI GT 70 APACHE PRO/Desktop/Kaggle/Baseline_forecast_train.csv')
baseline_forecast_test = pd.read_csv('C:/Users/MSI GT 70 APACHE PRO/Desktop/Kaggle/Baseline_forecast_test.csv')
baseline_observation_test = pd.read_csv("Baseline_observation_test.csv")
#train_arpege_with_y = pd.read_csv("C:/Users/MSI GT 70 APACHE PRO/Desktop/Kaggle/train_arpege_with_y.csv")
arpege_test = pd.read_csv('C:/Users/MSI GT 70 APACHE PRO/Desktop/Kaggle/2D_arpege_Test.csv')
arpege_train = pd.read_csv('C:/Users/MSI GT 70 APACHE PRO/Desktop/Kaggle/2D_arpege_Train.csv')

In [ ]:
df_station_test = pd.read_csv("C:/Users/MSI GT 70 APACHE PRO/Desktop/Kaggle/XtestV1.csv")

In [ ]:
arpege_test = pd.read_csv('C:/Users/MSI GT 70 APACHE PRO/Desktop/Kaggle/2D_arpege_Test.csv')

In [ ]:
arpege_train = pd.read_csv('C:/Users/MSI GT 70 APACHE PRO/Desktop/Kaggle/2D_arpege_Train.csv')

In [ ]:
df_station_train["Id_"] = ["_".join(i.split('_')[:2]) for i in list(df_station_train['Id'])]
#merged_train = pd.merge(df_station_train,df_y_train,how="left",left_on="Id_", right_on="Id")
df_station_train['hour'] = [i.split('_')[-1] for i in df_station_train['Id']]
df_station_train['hour'] = df_station_train['hour'].astype(int)

df_station_train['jour'] = [i.split('_')[1] for i in df_station_train['Id']]
df_station_train['jour'] = df_station_train['jour'].astype(int)
df_station_train['id_day'] = df_station_train['jour'].astype(str) + '_' +  df_station_train['hour'].astype(str)
df_station_train['date_day'] = [i[:10] for i in df_station_train['date']]
dico_jour_id_jour = dict(zip(df_station_train.date_day,df_station_train.jour))
liste_jours = list(set(df_station_train['date_day']))

# ARPEGE TRAIN

In [ ]:
df_station_train['time'] = [i[:10] for i in  df_station_train['date'] ]
arpege_train['time'] = [i[:10] for i in arpege_train['time']]
arpege_grouped_max = arpege_train.groupby(['number_sta','time']).max().reset_index()

In [ ]:
arpege_grouped_max

,number_sta,time,latitude,longitude,valid_time,ws,p3031,u10,v10,t2m,d2m,r,tp,msl,lat_stat,long_stat
0,14047002,2016-01-01,49.296001,-0.742,2016-01-02 00:00:00,9.980528,204.83163,1.491405,9.481744,282.24950,280.94406,97.741310,1.790634,102354.440,49.275,-0.712
1,14047002,2016-01-02,49.296001,-0.742,2016-01-03 00:00:00,14.451673,258.03537,12.901043,13.615006,284.03790,282.52585,91.457214,5.294279,100873.040,49.275,-0.712
2,14047002,2016-01-03,49.296001,-0.742,2016-01-04 00:00:00,14.889302,240.44340,7.760571,14.850050,283.80005,282.27457,94.126434,9.308434,100827.555,49.275,-0.712
3,14047002,2016-01-04,49.296001,-0.742,2016-01-05 00:00:00,10.506044,225.28079,7.021052,8.364319,283.06012,280.17276,91.971570,0.085571,98842.695,49.275,-0.712
4,14047002,2016-01-05,49.296001,-0.742,2016-01-06 00:00:00,9.785484,234.94520,5.433606,8.317701,282.87430,280.55966,95.794980,0.784819,99133.266,49.275,-0.712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206045,95690001,2017-12-27,49.096001,1.858,2017-12-28 00:00:00,13.271514,312.75220,10.954030,9.205397,279.41217,278.91500,96.626620,5.989933,100062.340,49.108,1.831
206046,95690001,2017-12-28,49.096001,1.858,2017-12-29 00:00:00,6.668086,287.98257,6.340935,2.642888,277.89795,275.24365,99.772400,0.002456,100981.360,49.108,1.831
206047,95690001,2017-12-29,49.096001,1.858,2017-12-30 00:00:00,9.657598,275.01750,9.268430,7.256721,280.95178,279.14493,97.967970,5.904898,101080.945,49.108,1.831
206048,95690001,2017-12-30,49.096001,1.858,2017-12-31 00:00:00,9.016445,260.68286,8.511057,7.034806,285.54120,284.59732,96.755660,8.980215,100930.870,49.108,1.831


In [ ]:
# POUR QUAND ON PREND LE MAX DE TP


df = df_station_train
df['dd_'] = [-(abs(192-i)) for i in df['dd']]
df['hour'] = [i.split('_')[-1] for i in df['Id']]
df['hour'] = df['hour'].astype(int)
df["Id_"] = ["_".join(i.split('_')[:2]) for i in list(df['Id'])]
df.index = df['Id_']
del df['Id_']
print("hey")
df_grouped_count = df.groupby('Id_').count()
df_grouped_count_24 = df_grouped_count[df_grouped_count['td']==24]
df_all_hours = df.loc[df_grouped_count_24.index]
df_all_hours['Id_'] = df_all_hours.index
index_id = list(df_all_hours.columns).index('Id_')
index_hour = list(df_all_hours.columns).index('hour')
index_ff = list(df_all_hours.columns).index('ff')
index_t = list(df_all_hours.columns).index('t')
index_td = list(df_all_hours.columns).index('td')
index_hu= list(df_all_hours.columns).index('hu')
index_dd_ = list(df_all_hours.columns).index('dd_')
index_precip = list(df_all_hours.columns).index('precip')
dict_stations = {}
dict_heures = {}
for station_jour in set(df_all_hours.index):
    dict_stations[station_jour] = np.zeros([6,24]).astype('f')
    


for i in tqdm(df_all_hours.values):
    dict_stations[i[index_id]][0][i[index_hour]] = i[index_ff]
    dict_stations[i[index_id]][1][i[index_hour]] = i[index_t]
    dict_stations[i[index_id]][2][i[index_hour]] = i[index_td]
    dict_stations[i[index_id]][3][i[index_hour]] = i[index_hu]
    dict_stations[i[index_id]][4][i[index_hour]] = i[index_dd_]
    dict_stations[i[index_id]][5][i[index_hour]] = i[index_precip]
new_def_colums_hours = pd.DataFrame([[station]+[a for i in dict_stations[station] for a in i ] for station in set(df_all_hours.index)])
new_def_colums_hours_merged = pd.merge(new_def_colums_hours,df_y_train,left_on =0,right_on='Id')

hey


100%|████████████████████████████████████████████████████████████████████| 4409064/4409064 [00:28<00:00, 156484.70it/s]


In [ ]:
merged_all = pd.merge(new_def_colums_hours_merged,arpege_grouped_max,left_on = ['number_sta','date'],right_on = ['number_sta','time'])
del merged_all['date']
del merged_all['number_sta']
del merged_all['Id']
del merged_all['valid_time']

#del merged_all['time']
merged_all = merged_all.dropna()
train = merged_all
train['month'] = [int(i[5:7]) for i in train['time']]
del train['time']
#train = pd.read_csv("train_with_arome.csv")
X_train = train[list(train.columns[1:145])+['tp','month']] #mettre ici colonnestrain
Y_train = train['Ground_truth']
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)


x_train = x_train.astype('f')
x_test = x_test.astype('f')
y_train = y_train.astype('f')
y_test = y_test.astype('f')
x_train = ((x_train - x_train.mean())/x_train.std())
x_test = ((x_test - x_test.mean())/x_test.std())

In [ ]:
X_train = (X_train-X_train.mean())/X_train.std()

# ARPEGE TEST

In [ ]:
arpege_test['Id_'] = arpege_test['number_sta'].astype(str) + '_' + arpege_test['Id']
arpege_test = arpege_test.groupby('Id_').mean().reset_index()
arpege_test['new_id'] = ["_".join(i.split('_')[:2]) for i in list(arpege_test['Id_'])]
df_station_arpege_merge = pd.merge(df_station_test,arpege_test,left_on ='Id',right_on ='Id_')
arpege_test_grouped_max = arpege_test.groupby('new_id').max()
arpege_test_grouped_max = arpege_test_grouped_max.reset_index()

In [ ]:
#POUR XTEST

#TEST

df = df_station_test.copy()
df['dd_'] = [-(abs(192-i)) for i in df['dd']]
df['hour'] = [i.split('_')[-1] for i in df['Id']]
df['hour'] = df['hour'].astype(int)
df["Id_"] = ["_".join(i.split('_')[:2]) for i in list(df['Id'])]
df.index = df['Id_']
del df['Id_']
df_grouped_count = df.groupby('Id_').count()
df_grouped_count_24 = df_grouped_count[df_grouped_count['td']==24]
df_all_hours = df.loc[df_grouped_count_24.index]
df_all_hours['Id_'] = df_all_hours.index
index_id =list(df_all_hours.columns).index('Id_')
index_hour = list(df_all_hours.columns).index('hour')
index_ff = list(df_all_hours.columns).index('ff')
index_t = list(df_all_hours.columns).index('lt')
index_td = list(df_all_hours.columns).index('td')
index_hu= list(df_all_hours.columns).index('hu')
index_dd_ = list(df_all_hours.columns).index('dd_')
index_precip = list(df_all_hours.columns).index('precip')
dict_stations = {}
dict_heures = {}
for station_jour in set(df_all_hours.index):
    dict_stations[station_jour] = np.zeros([6,24]).astype('f')
    


for i in tqdm(df_all_hours.values):
    dict_stations[i[index_id]][0][i[index_hour]] = i[index_ff]
    dict_stations[i[index_id]][1][i[index_hour]] = i[index_t]
    dict_stations[i[index_id]][2][i[index_hour]] = i[index_td]
    dict_stations[i[index_id]][3][i[index_hour]] = i[index_hu]
    dict_stations[i[index_id]][4][i[index_hour]] = i[index_dd_]
    dict_stations[i[index_id]][5][i[index_hour]] = i[index_precip]
new_def_colums_hours = pd.DataFrame([[station]+[a for i in dict_stations[station] for a in i ] for station in set(df_all_hours.index)])
new_def_colums_hours_merged = new_def_colums_hours
new_def_colums_hours_merged_prediction = new_def_colums_hours_merged.copy()
#new_def_colums_hours_merged_prediction
#del new_def_colums_hours_merged_prediction['number_sta']
#del new_def_colums_hours_merged_prediction['date']
#del new_def_colums_hours_merged_prediction['Id']
#new_def_colums_hours_merged_prediction
train = pd.merge(new_def_colums_hours_merged_prediction,arpege_test_grouped_max,left_on = 0,right_on ='new_id')
# X_train = train[list(train.columns[1:25])+[48,72,96,120,144]+['Prediction']] #mettre ici colonnestrain
#X_train = (X_train-X_train.mean())/X_train.std

100%|████████████████████████████████████████████████████████████████████| 2304288/2304288 [00:15<00:00, 146179.98it/s]


In [ ]:
#train = pd.merge(new_def_colums_hours_merged_prediction,arpege_test_grouped_max,left_on = 0,right_on ='new_id')

In [ ]:
xtest= train[list(train.columns[1:145])+['tp','time']]

In [ ]:
xtest = (xtest -xtest.mean())/xtest.std()

In [ ]:
ytest = train[0].copy()

In [ ]:
train = pd.merge(new_def_colums_hours_merged_prediction,arpege_test_grouped_max,left_on = 0,right_on ='new_id')


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,Prediction,new_id,Id_,latitude,longitude,valid_time,time,ws,p3031,u10,v10,t2m,d2m,r,tp,msl,lat_stat,long_stat,number_sta
0,29113001_232,1.79,1.69,1.74,1.75,1.62,1.93,2.44,3.97,4.44,3.84,3.84,4.36,5.64,5.86,6.58,6.84,6.76,6.37,5.92,4.60,2.76,1.26,0.98,1.33,288.040009,287.570007,286.940002,286.839996,286.619995,287.730011,289.100006,290.390015,292.239990,294.040009,295.230011,294.730011,294.420013,295.339996,295.369995,294.859985,294.450012,293.890015,292.660004,289.709991,288.170013,287.239990,286.010010,285.720001,286.829987,286.459991,286.049988,285.899994,286.019989,286.779999,287.799988,287.619995,287.809998,287.959991,288.019989,288.410004,288.119995,286.989990,286.440002,285.899994,286.029999,285.700012,285.540009,286.000000,286.269989,286.049988,285.929993,285.809998,96.000000,96.099998,96.800003,97.400002,98.000000,97.900002,91.800003,86.099998,83.699997,79.000000,73.300003,71.000000,70.800003,62.500000,61.200001,60.700001,61.700001,63.599998,67.400002,77.000000,88.300003,93.599998,95.800003,97.099998,-139.0,-148.0,-149.0,-151.0,-147.0,-163.0,-2.0,-148.0,-161.0,-162.0,-166.0,-169.0,-172.0,-169.0,-163.0,-163.0,-156.0,-158.0,-164.0,-162.0,-162.0,-163.0,-151.0,-170.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009713,29113001_232,29113001_232_9,48.695999,-3.742,NaN,7,8.130393,176.92482,-0.032481,0.619881,295.39050,287.64548,97.714066,0.000779,102211.695,48.663,-3.693,29113001
1,37213003_40,1.59,2.16,2.22,1.81,2.04,0.91,1.98,2.41,1.88,1.80,2.18,2.85,2.84,3.34,3.51,4.17,3.81,3.87,3.24,2.25,1.97,1.77,1.46,1.65,288.709991,287.720001,287.160004,286.910004,286.040009,286.799988,290.179993,294.000000,296.309998,297.739990,298.649994,299.309998,299.359985,300.239990,300.459991,300.589996,301.160004,300.109985,298.899994,294.989990,292.600006,291.510010,290.950012,289.820007,285.829987,285.690002,285.190002,284.739990,284.290009,285.209991,285.790009,286.269989,286.079987,286.059998,286.109985,285.660004,285.329987,285.279999,284.929993,284.160004,284.100006,283.519989,283.320007,283.910004,283.950012,283.959991,284.480011,285.290009,83.099998,87.599998,88.000000,86.699997,89.099998,90.000000,75.500000,61.400002,52.599998,48.200001,45.799999,42.799999,41.700001,39.500000,38.099998,35.900002,34.599998,35.400002,37.599998,49.500000,57.400002,61.500000,65.900002,74.599998,-132.0,-133.0,-134.0,-139.0,-139.0,-138.0,-52.0,-75.0,-32.0,-23.0,-134.0,-75.0,-104.0,-106.0,-134.0,-113.0,-134.0,-117.0,-113.0,-100.0,-92.0,-100.0,-73.0,-95.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,37213003_40,37213003_40_9,47.596001,0.458,NaN,8,4.346568,316.27820,1.037105,0.916696,301.63956,286.15906,65.941980,0.003683,101809.260,47.615,0.464,37213003
2,29075001_113,7.40,6.99,5.74,3.90,4.61,5.82,6.37,5.81,5.87,5.37,6.26,7.88,5.79,4.45,4.63,5.23,5.76,5.54,4.61,4.54,5.27,5.36,6.07,6.65,292.190002,292.109985,291.980011,291.339996,290.609985,290.929993,291.239990,291.959991,292.910004,292.970001,292.809998,292.369995,292.589996,293.109985,293.070007,292.880005,292.839996,292.769989,292.140015,291.950012,291.779999,291.380005,291.290009,290.750000,280.489990,280.630005,281.260010,282.070007,283.369995,285.630005,287.200012,288.140015,289.529999,290.109985,290.709991,291.390015,291.440002,291.220001,291.220001,290.049988,289.510010,289.609985,290.010010,290.040009,290.040009,289.980011,290.190002,289.880005,46.599998,47.400002,49.799999,54.799999,62.599998,71.199997,77.300003,78.400002,80.90

In [ ]:
new_def_colums_hours.to_csv("finale_test_with_arpege.csv")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168
0,29113001_232,1.79,1.69,1.74,1.75,1.62,1.93,2.44,3.97,4.44,3.84,3.84,4.36,5.64,5.86,6.58,6.84,6.76,6.37,5.92,4.60,2.76,1.26,0.98,1.33,288.040009,287.570007,286.940002,286.839996,286.619995,287.730011,289.100006,290.390015,292.239990,294.040009,295.230011,294.730011,294.420013,295.339996,295.369995,294.859985,294.450012,293.890015,292.660004,289.709991,288.170013,287.239990,286.010010,285.720001,286.829987,286.459991,286.049988,285.899994,286.019989,286.779999,287.799988,287.619995,287.809998,287.959991,288.019989,288.410004,288.119995,286.989990,286.440002,285.899994,286.029999,285.700012,285.540009,286.000000,286.269989,286.049988,285.929993,285.809998,96.000000,96.099998,96.800003,97.400002,98.000000,97.900002,91.800003,86.099998,83.699997,79.000000,73.300003,71.000000,70.800003,62.500000,61.200001,60.700001,61.700001,63.599998,67.400002,77.000000,88.300003,93.599998,95.800003,97.099998,-139.0,-148.0,-149.0,-151.0,-147.0,-163.0,-2.0,-148.0,-161.0,-162.0,-166.0,-169.0,-172.0,-169.0,-163.0,-163.0,-156.0,-158.0,-164.0,-162.0,-162.0,-163.0,-151.0,-170.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.000000,0.000223,0.000390,0.000390,0.000390,0.000390,0.000390,0.000779,0.000390,0.000390,0.000390,0.000390,0.000390,0.000390,0.000390,0.000390,0.000390,0.000390,0.000390,0.000390,0.000390,0.000390,0.000390
1,37213003_40,1.59,2.16,2.22,1.81,2.04,0.91,1.98,2.41,1.88,1.80,2.18,2.85,2.84,3.34,3.51,4.17,3.81,3.87,3.24,2.25,1.97,1.77,1.46,1.65,288.709991,287.720001,287.160004,286.910004,286.040009,286.799988,290.179993,294.000000,296.309998,297.739990,298.649994,299.309998,299.359985,300.239990,300.459991,300.589996,301.160004,300.109985,298.899994,294.989990,292.600006,291.510010,290.950012,289.820007,285.829987,285.690002,285.190002,284.739990,284.290009,285.209991,285.790009,286.269989,286.079987,286.059998,286.109985,285.660004,285.329987,285.279999,284.929993,284.160004,284.100006,283.519989,283.320007,283.910004,283.950012,283.959991,284.480011,285.290009,83.099998,87.599998,88.000000,86.699997,89.099998,90.000000,75.500000,61.400002,52.599998,48.200001,45.799999,42.799999,41.700001,39.500000,38.099998,35.900002,34.599998,35.400002,37.599998,49.500000,57.400002,61.500000,65.900002,74.599998,-132.0,-133.0,-134.0,-139.0,-139.0,-138.0,-52.0,-75.0,-32.0,-23.0,-134.0,-75.0,-104.0,-106.0,-134.0,-113.0,-134.0,-117.0,-113.0,-100.0,-92.0,-100.0,-73.0,-95.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000065,0.000129,0.000129,0.000129,0.000129,0.001228,0.002973,0.003425,0.003425
2,29075001_113,7.40,6.99,5.74,3.90,4.61,5.82,6.37,5.81,5.87,5.37,6.26,7.88,5.79,4.45,4.63,5.23,5.76,5.54,4.61,4.54,5.27,5.36,6.07,6.65,292.190002,292.109985,291.980011,291.339996,290.609985,290.929993,291.239990,291.959991,292.910004,292.970001,292.809998,292.369995,292.589996,293.109985,293.070007,292.880005,292.839996,292.769989,292.140015,291.950012,291.779999,291.380005,291.290009,290.750000,280.489990,280.630005,281.260010,282.070007,283.369995,285.630005,287.200012,288.140015,289.529999,290.109985,290.709991,291.390015,291.440002,291.220001,291.220001,290.049988,289.510010,289.609985,290.010010,290.040009,290.040009,289.980011,290.190002,289.880005,46.599998,47.400002,49.79

In [ ]:
def score(predictions,labels):
    return np.mean([np.abs((labels[i]-predictions[i])/(labels[i]+1)) for i in range(len(predictions))])*100

In [ ]:
#!pip install tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation,Flatten,BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, InputLayer 
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import scipy
import scipy.ndimage
import numpy as np
from keras import regularizers
from tensorflow.keras import backend as K
def custom_loss_function(y_true, y_pred):

    return tf.math.reduce_mean(tf.abs((y_true+1)-(y_pred+1))/(y_true+1))*100
# import regularizer
from keras.regularizers import l1
# instantiate regularizer
reg = l1(0.001)
from keras.regularizers import l2

In [ ]:
def getSimplePerceptron(nb_classes=2): 
    model = Sequential()
    model.add(InputLayer(input_shape=(145)))
    
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu',))
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu',))
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu',))



	#model.add(Dense(1, kernel_initializer='normal'))
    model.add(Dense(1, activation='relu'))
    model.compile(loss=custom_loss_function, optimizer="adam", metrics=custom_loss_function)
    return model

model=getSimplePerceptron()
model=getSimplePerceptron()
batch_size = 256
epochs=20
model.fit(np.array(x_train).astype('f'), np.array(y_train).astype('f'), batch_size=batch_size, epochs=epochs,  verbose=1)
pred = model.predict(np.array(x_test).astype('f'))
score(pred,np.array(y_test))

Epoch 1/20
329/329 [==============================] - 8s 22ms/step - loss: 28.7986 - custom_loss_function: 28.7996
Epoch 2/20
329/329 [==============================] - 8s 23ms/step - loss: 25.6152 - custom_loss_function: 25.6123
Epoch 3/20
329/329 [==============================] - 8s 23ms/step - loss: 24.7688 - custom_loss_function: 24.7681
Epoch 4/20
329/329 [==============================] - 7s 22ms/step - loss: 24.3201 - custom_loss_function: 24.3213
Epoch 5/20
329/329 [==============================] - 8s 23ms/step - loss: 23.9835 - custom_loss_function: 23.9849
Epoch 6/20
329/329 [==============================] - 8s 24ms/step - loss: 23.8292 - custom_loss_function: 23.8289
Epoch 7/20
329/329 [==============================] - 8s 25ms/step - loss: 23.5375 - custom_loss_function: 23.5385
Epoch 8/20
329/329 [==============================] - 8s 23ms/step - loss: 23.4110 - custom_loss_function: 23.4099
Epoch 9/20
329/329 [==============================] - 8s 23ms/step - loss: 23.03

22.989723086357117

In [ ]:
def getSimplePerceptron(nb_classes=2): 
    model = Sequential()
    model.add(InputLayer(input_shape=(145)))
    
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu',))
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu',))
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu',))
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu',))



	#model.add(Dense(1, kernel_initializer='normal'))
    model.add(Dense(1, activation='relu'))
    model.compile(loss=custom_loss_function, optimizer="adam", metrics=custom_loss_function)
    return model

model=getSimplePerceptron()
model=getSimplePerceptron()
batch_size = 256
epochs=20
model.fit(np.array(x_train).astype('f'), np.array(y_train).astype('f'), batch_size=batch_size, epochs=epochs,  verbose=1)
pred = model.predict(np.array(x_test).astype('f'))
score(pred,np.array(y_test))

Epoch 1/20
329/329 [==============================] - 6s 15ms/step - loss: 28.2403 - custom_loss_function: 28.2416
Epoch 2/20
329/329 [==============================] - 5s 15ms/step - loss: 25.3624 - custom_loss_function: 25.3617
Epoch 3/20
329/329 [==============================] - 5s 15ms/step - loss: 24.5588 - custom_loss_function: 24.5580
Epoch 4/20
329/329 [==============================] - 5s 15ms/step - loss: 24.1941 - custom_loss_function: 24.1972
Epoch 5/20
329/329 [==============================] - 5s 15ms/step - loss: 23.9440 - custom_loss_function: 23.9420
Epoch 6/20
329/329 [==============================] - 5s 16ms/step - loss: 23.7032 - custom_loss_function: 23.7085
Epoch 7/20
329/329 [==============================] - 5s 16ms/step - loss: 23.4444 - custom_loss_function: 23.4450
Epoch 8/20
329/329 [==============================] - 5s 16ms/step - loss: 23.1531 - custom_loss_function: 23.1512
Epoch 9/20
329/329 [==============================] - 5s 16ms/step - loss: 22.98

22.764596343040466

In [ ]:
def getSimplePerceptron(nb_classes=2): 
    model = Sequential()
    model.add(InputLayer(input_shape=(145)))
    
    model.add(Dense(512, activation='relu',))
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu',))
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu',))
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu',))
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu',))
    



	#model.add(Dense(1, kernel_initializer='normal'))
    model.add(Dense(1, activation='relu'))
    model.compile(loss=custom_loss_function, optimizer="adam", metrics=custom_loss_function)
    return model

model=getSimplePerceptron()
model=getSimplePerceptron()
batch_size = 256
epochs=20
model.fit(np.array(x_train).astype('f'), np.array(y_train).astype('f'), batch_size=batch_size, epochs=epochs,  verbose=1)
pred = model.predict(np.array(x_test).astype('f'))
score(pred,np.array(y_test))

Epoch 1/15
329/329 [==============================] - 7s 17ms/step - loss: 28.0977 - custom_loss_function: 28.0947
Epoch 2/15
329/329 [==============================] - 6s 17ms/step - loss: 25.4490 - custom_loss_function: 25.4528
Epoch 3/15
329/329 [==============================] - 6s 18ms/step - loss: 24.6879 - custom_loss_function: 24.6838
Epoch 4/15
329/329 [==============================] - 6s 19ms/step - loss: 24.2396 - custom_loss_function: 24.2394
Epoch 5/15
329/329 [==============================] - 6s 20ms/step - loss: 23.9244 - custom_loss_function: 23.9242
Epoch 6/15
329/329 [==============================] - 6s 19ms/step - loss: 23.8556 - custom_loss_function: 23.8544
Epoch 7/15
329/329 [==============================] - 6s 19ms/step - loss: 23.5751 - custom_loss_function: 23.5735
Epoch 8/15
329/329 [==============================] - 6s 19ms/step - loss: 23.3270 - custom_loss_function: 23.3288
Epoch 9/15
329/329 [==============================] - 6s 19ms/step - loss: 23.08

23.23332577943802

In [ ]:
def getSimplePerceptron(nb_classes=2): 
    model = Sequential()
    model.add(InputLayer(input_shape=(146)))
    
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu',))
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu',))
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu',))



	#model.add(Dense(1, kernel_initializer='normal'))
    model.add(Dense(1, activation='relu'))
    model.compile(loss=custom_loss_function, optimizer="adam", metrics=custom_loss_function)
    return model

model=getSimplePerceptron()
model=getSimplePerceptron()
batch_size = 256
epochs=20
model.fit(np.array(x_train).astype('f'), np.array(y_train).astype('f'), batch_size=batch_size, epochs=epochs,  verbose=1)
pred = model.predict(np.array(x_test).astype('f'))
score(pred,np.array(y_test))

Epoch 1/20
329/329 [==============================] - 5s 13ms/step - loss: 28.4545 - custom_loss_function: 28.4577
Epoch 2/20
329/329 [==============================] - 4s 13ms/step - loss: 25.9025 - custom_loss_function: 25.9017
Epoch 3/20
329/329 [==============================] - 4s 13ms/step - loss: 24.8019 - custom_loss_function: 24.8021
Epoch 4/20
329/329 [==============================] - 4s 13ms/step - loss: 24.3828 - custom_loss_function: 24.3839
Epoch 5/20
329/329 [==============================] - 4s 14ms/step - loss: 23.9615 - custom_loss_function: 23.9662
Epoch 6/20
329/329 [==============================] - 5s 15ms/step - loss: 23.7356 - custom_loss_function: 23.7349
Epoch 7/20
329/329 [==============================] - 4s 13ms/step - loss: 23.4595 - custom_loss_function: 23.4612
Epoch 8/20
329/329 [==============================] - 4s 13ms/step - loss: 23.1391 - custom_loss_function: 23.1376
Epoch 9/20
329/329 [==============================] - 4s 14ms/step - loss: 22.94

22.74923473596573

In [ ]:
def getSimplePerceptron(nb_classes=2): 
    model = Sequential()
    model.add(InputLayer(input_shape=(146)))
    
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu',))
    model.add(BatchNormalization())
    model.add(Dense(512, activation='relu',))
    model.add(BatchNormalization())
    model.add(Dense(256, activation='relu',))




	#model.add(Dense(1, kernel_initializer='normal'))
    model.add(Dense(1, activation='relu'))
    model.compile(loss=custom_loss_function, optimizer="adam", metrics=custom_loss_function)
    return model

model=getSimplePerceptron()
model=getSimplePerceptron()
batch_size = 128
epochs=20
model.fit(np.array(x_train).astype('f'), np.array(y_train).astype('f'), batch_size=batch_size, epochs=epochs,  verbose=1)
pred = model.predict(np.array(x_test).astype('f'))
score(pred,np.array(y_test))

Epoch 1/20
658/658 [==============================] - 7s 9ms/step - loss: 28.2162 - custom_loss_function: 28.2187
Epoch 2/20
658/658 [==============================] - 6s 9ms/step - loss: 25.3425 - custom_loss_function: 25.3455
Epoch 3/20
658/658 [==============================] - 6s 9ms/step - loss: 24.8052 - custom_loss_function: 24.8082
Epoch 4/20
658/658 [==============================] - 6s 9ms/step - loss: 24.5600 - custom_loss_function: 24.5599
Epoch 5/20
658/658 [==============================] - 6s 9ms/step - loss: 24.2082 - custom_loss_function: 24.2091
Epoch 6/20
658/658 [==============================] - 6s 9ms/step - loss: 23.9161 - custom_loss_function: 23.9164
Epoch 7/20
658/658 [==============================] - 6s 9ms/step - loss: 23.7019 - custom_loss_function: 23.6988
Epoch 8/20
658/658 [==============================] - 6s 9ms/step - loss: 23.4835 - custom_loss_function: 23.4854
Epoch 9/20
658/658 [==============================] - 6s 9ms/step - loss: 23.2445 - cust

22.71660715341568

In [ ]:
def getSimplePerceptron(nb_classes=2): 
    model = Sequential()
    model.add(InputLayer(input_shape=(146)))
    
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(64, activation='relu',))
    model.add(BatchNormalization())
    model.add(Dense(64, activation='relu',))
    model.add(BatchNormalization())
    model.add(Dense(64, activation='relu',))
    model.add(BatchNormalization())
    model.add(Dense(64, activation='relu',))




	#model.add(Dense(1, kernel_initializer='normal'))
    model.add(Dense(1, activation='relu'))
    model.compile(loss=custom_loss_function, optimizer="adam", metrics=custom_loss_function)
    return model

model=getSimplePerceptron()

batch_size = 256
epochs=20
model.fit(np.array(X_train).astype('f'), np.array(Y_train).astype('f'), batch_size=batch_size, epochs=epochs,  verbose=1)
# pred = model.predict(np.array(x_test).astype('f'))
# score(pred,np.array(y_test))

Epoch 1/20
548/548 [==============================] - 10s 5ms/step - loss: 28.0644 - custom_loss_function: 28.0638
Epoch 2/20
548/548 [==============================] - 3s 5ms/step - loss: 25.1817 - custom_loss_function: 25.1831
Epoch 3/20
548/548 [==============================] - 3s 5ms/step - loss: 24.5796 - custom_loss_function: 24.5798
Epoch 4/20
548/548 [==============================] - 3s 5ms/step - loss: 24.2069 - custom_loss_function: 24.2075
Epoch 5/20
548/548 [==============================] - 3s 5ms/step - loss: 23.8969 - custom_loss_function: 23.8961
Epoch 6/20
548/548 [==============================] - 3s 5ms/step - loss: 23.5729 - custom_loss_function: 23.5727
Epoch 7/20
548/548 [==============================] - 3s 5ms/step - loss: 23.3636 - custom_loss_function: 23.3644
Epoch 8/20
548/548 [==============================] - 3s 5ms/step - loss: 23.1228 - custom_loss_function: 23.1229
Epoch 9/20
548/548 [==============================] - 3s 5ms/step - loss: 22.8872 - cus

# PREDICTION FINALE

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score

In [ ]:
predictions_finales = model.predict(np.array(xtest).astype('f'))  #PREDICTIONS TEST

In [ ]:
y_train2 = (y_train.copy()>0.2).astype(int)
y_test2 = (y_test.copy()>0.2).astype(int)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.datasets import make_classification

clf = RandomForestClassifier(max_depth=30, random_state=0)
clf.fit(x_train, y_train2)
print(recall_score(y_test2,clf.predict(x_test)))
accuracy_score(y_test2,clf.predict(x_test))

# COMBINAISON PREDICTION PLUIE ET REGRESSION

In [ ]:
def adapte_score(preds,pred_pluie) :
    cpt = 0
    for i in range(preds.shape[0]):
        if pred_pluie[i]==0 :
            preds[i] =  preds[i]/2
            cpt+=1
    return preds
pred_ad = adapte_score(predictions_finales.copy(),clf.predict(xtest))

C:\Users\MSI GT 70 APACHE PRO\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


35150


In [ ]:
df_predictions = pd.DataFrame(predictions_finales,columns = ['Prediction'])

In [ ]:
df_predictions['Id'] = ytest

In [ ]:
df_prediction_merged = pd.merge(df_predictions,baseline_observation_test,left_on ='Id',right_on ='Id',how = 'right')

In [ ]:
df_final = df_prediction_merged.fillna(0)

In [ ]:
df_final = df_final[['Prediction_x','Id']].copy()

In [ ]:
df_final['Prediction'] = df_final['Prediction_x']+1

In [ ]:
del df_final['Prediction_x']

In [ ]:
df_final.to_csv('dernieres_predictions2.csv',index = False)

In [ ]:
df_final

,Id,Prediction
0,14066001_149,1.000000
1,14126001_149,1.231701
2,14137001_149,1.000000
3,14216001_149,1.000000
4,14296001_149,1.220735
...,...,...
85135,86137003_293,1.000000
85136,86165005_293,1.194200
85137,86273001_293,1.192044
85138,91200002_293,1.000000


# MOYENNE DE NOS MODELES

In [ ]:
d = pd.read_csv('moyennepred.csv')

In [ ]:
d['Prediction'] = (d['Prediction1']+d['Prediction2']+d['Prediction3'])/3

In [ ]:
d = d[['Prediction','Id']]